In [3]:
import copy
import networkx as nx
import pandas as pd 
from collections import defaultdict, OrderedDict

from bondnet.utils import to_path, int_atom, xyz2mol
from bondnet.core.molwrapper import rdkit_mol_to_wrapper_mol, create_wrapper_mol_from_atoms_and_bonds
from pymatgen.analysis.graphs import _isomorphic
from bondnet.core.reaction_collection import get_atom_bond_mapping
from bondnet.dataset.mg_barrier import process_species

path_mg_data = "/home/santiagovargas/Documents/Dataset/mg_dataset/"
path_json = path_mg_data + "20220613_reaction_data.json"
mg_df = pd.read_json(path_json)

/home/santiagovargas/anaconda3/envs/bondnet/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using backend: pytorch


In [27]:
row = mg_df.iloc[100]

species_reactant = [int_atom(i['name']) for i in row['reactant_molecule_graph']["molecule"]["sites"]]
species_products = [int_atom(i['name']) for i in row['product_molecule_graph']["molecule"]["sites"]]
coords_reactant = [i["xyz"] for i in row['reactant_molecule_graph']["molecule"]["sites"]]
coords_products = [i["xyz"] for i in row['product_molecule_graph']["molecule"]["sites"]]
charge = row['charge']
id = str(row["reactant_id"])
free_energy = row['product_free_energy']

reactant_mol = xyz2mol(atoms = species_reactant, 
    coordinates = coords_reactant, 
    charge = charge,
    )
reactant_wrapper = rdkit_mol_to_wrapper_mol(reactant_mol[0],
    charge = charge,
    free_energy=free_energy, 
    identifier = id
    )
print(reactant_wrapper.bonds) 
print()

#print(species_reactant)
#print(species_products)

xyz_list = []
for i in reactant_mol[0].GetAtoms():
    xyz_list.append(i.GetAtomicNum())
print(xyz_list)    

rxn_alt_double_product = process_species(row)

#print(list(reactant_wrapper.bonds.keys()))
print(row['reactant_bonds_nometal'])

{(0, 2): {}, (0, 3): {}, (0, 7): {}, (0, 16): {}, (1, 2): {}, (2, 10): {}, (2, 13): {}, (3, 4): {}, (3, 5): {}, (3, 6): {}, (7, 8): {}, (7, 9): {}, (10, 11): {}, (10, 12): {}, (13, 14): {}, (13, 15): {}}

[6, 1, 6, 6, 1, 1, 1, 7, 1, 1, 7, 8, 8, 7, 1, 1, 9, 9]
[[0, 3], [0, 2], [0, 16], [0, 7], [1, 2], [2, 13], [2, 10], [3, 4], [3, 5], [3, 6], [7, 9], [7, 8], [10, 11], [10, 12], [13, 14], [13, 15]]


In [5]:

# 101 is single product molecule...100 is two products
# todo: fill in bonds not found by rdkit w/evan's table 
# todo: 
#row = mg_df.iloc[101]
#rxn_alt_double_product = process_species(row)

#species_reactant = [int_atom(i['name']) for i in row['reactant_molecule_graph']["molecule"]["sites"]]
#species_products = [int_atom(i['name']) for i in row['product_molecule_graph']["molecule"]["sites"]]
#coords_reactant = [i["xyz"] for i in row['reactant_molecule_graph']["molecule"]["sites"]]
#coords_products = [i["xyz"] for i in row['product_molecule_graph']["molecule"]["sites"]]

#print(rxn_alt_double_product.reactants)
#print(rxn_alt_double_product.products)


#print(atom_mapping_test(rxn_alt_double_product, row))
#print(nx_graph_atom_mapping(rxn_alt_double_product.reactants[0].graph, rxn_alt_double_product.products.graph))


In [41]:
works_count = 0
works_single_product = 0
single_product_count = 0
double_product_count = 0 
works_double_product = 0 
from tqdm import tqdm 

for i in tqdm(range(200)):
    row = mg_df.iloc[i]
    """g_prod = nx.Graph(row["product_bonds"])
    g_react = nx.Graph(row["reactant_bonds"])
    g_prod_2 = nx.Graph(row["product_bonds_nometal"])
    g_react_2 = nx.Graph(row["reactant_bonds_nometal"])
    if(nx_graph_atom_mapping(g_prod, g_react) == None):
        if(nx_graph_atom_mapping(g_prod_2, g_react_2) == None):
            pass
        else: works_count += 1 
    else: works_count += 1"""
    species_reactant = [int_atom(i['name']) for i in row['reactant_molecule_graph']["molecule"]["sites"]]
    species_products = [int_atom(i['name']) for i in row['product_molecule_graph']["molecule"]["sites"]]
    coords_reactant = [i["xyz"] for i in row['reactant_molecule_graph']["molecule"]["sites"]]
    coords_products = [i["xyz"] for i in row['product_molecule_graph']["molecule"]["sites"]]

    r = create_wrapper_mol_from_atoms_and_bonds(
        species_reactant, coords_reactant, row["reactant_bonds"], charge=0
    )
    p = create_wrapper_mol_from_atoms_and_bonds(
        species_products, coords_products, row["product_bonds"], charge=0
    )
    r2 = create_wrapper_mol_from_atoms_and_bonds(
        species_reactant, coords_reactant, row["reactant_bonds_nometal"], charge=0
    )
    p2 = create_wrapper_mol_from_atoms_and_bonds(
        species_products, coords_products, row["product_bonds_nometal"], charge=0
    )
  
    rxn_alt_double_product = []

    if(type(rxn_alt_double_product) == list):
        products = p
        product_s = copy.deepcopy(products)
        #original.break_edge(bond[0], bond[1], allow_reverse=True)
        components = nx.weakly_connected_components(product_s.graph)
        subgraphs = [product_s.graph.subgraph(c) for c in components]
        subgraphs_count = int(len(subgraphs))

    else: 
        products = rxn_alt_double_product.products
        subgraphs_count = int(len(products))

    if(subgraphs_count==2):double_product_count+=1
    else: 
        single_product_count+=1
        mapping = 

    """
    if(subgraphs_count == 2):
        if(nx_graph_atom_mapping(r.mol_graph.graph, p.mol_graph.graph) == None):
            if(nx_graph_atom_mapping(r2.mol_graph.graph, p2.mol_graph.graph) == None):
                if(nx_graph_atom_mapping(r.mol_graph.graph, p2.mol_graph.graph) == None):
                    if(nx_graph_atom_mapping(r2.mol_graph.graph, p.mol_graph.graph) == None):
                        pass
                    else: 
                        works_count += 1
                        works_double_product+=1
                else: 
                    works_count += 1
                    works_double_product+=1
            else: 
                works_count += 1 
                works_double_product+=1
        else: 
            works_count += 1
            works_double_product+=1
        if(nx_graph_atom_mapping(r.mol_graph.graph, p.mol_graph.graph) == None):
            if(nx_graph_atom_mapping(r2.mol_graph.graph, p2.mol_graph.graph) == None):
                if(nx_graph_atom_mapping(r.mol_graph.graph, p2.mol_graph.graph) == None):
                    if(nx_graph_atom_mapping(r2.mol_graph.graph, p.mol_graph.graph) == None):
                        pass
                    else: 
                        works_count += 1
                        works_double_product+=1
                else: 
                    works_count += 1
                    works_double_product+=1
            else: 
                works_count += 1 
                works_double_product+=1
        else: 
            works_count += 1
            works_double_product+=1
   
    else:"""
    if(nx_graph_atom_mapping(r.mol_graph.graph, p.mol_graph.graph) == None):
        if(nx_graph_atom_mapping(r2.mol_graph.graph, p2.mol_graph.graph) == None):
            if(nx_graph_atom_mapping(r.mol_graph.graph, p2.mol_graph.graph) == None):
                if(nx_graph_atom_mapping(r2.mol_graph.graph, p.mol_graph.graph) == None):
                    pass
                else: 
                    works_count += 1
                    if(subgraphs_count == 1): works_single_product+=1
                    else: works_double_product+=1
            else: 
                works_count += 1
                if(subgraphs_count == 1): works_single_product+=1
                else: works_double_product+=1
        else: 
            works_count += 1 
            if(subgraphs_count == 1): works_single_product+=1
            else: works_double_product+=1
    else: 
        works_count += 1
        if(subgraphs_count == 1): works_single_product+=1
        else: works_double_product+=1


100%|██████████| 200/200 [00:00<00:00, 260.51it/s]


In [42]:
print("works for {} double product reactions".format(works_double_product))
print("works for {} single product reactions".format(works_single_product))
print("works overall: {:.2f}%".format(works_count/(double_product_count + single_product_count)))
print("count 2: {:.2f}%".format(works_double_product/double_product_count))
print("count 1: {:.2f}%".format(works_single_product/ single_product_count))

works for 21 double product reactions
works for 39 single product reactions
works overall: 0.30%
count 2: 0.28%
count 1: 0.31%


In [10]:
print("works for {} double product reactions".format(works_double_product))
print("works for {} single product reactions".format(works_single_product))
print("works overall: {:.2f}%".format(works_count/(double_product_count + single_product_count)))
print("count 2: {:.2f}%".format(works_double_product/double_product_count))
print("count 1: {:.2f}%".format(works_single_product/ single_product_count))

works for 21 double product reactions
works for 39 single product reactions
works overall: 0.30%
count 2: 0.28%
count 1: 0.31%


In [42]:
element_dict = {"H": 1, "Li": 3, "Be": 4, "B": 5, "C": 6, "N": 7, "O": 8, 
"F": 9, 'Ca': 20, 'Na': 11, 'Cl': 17, 'P':15, 'Mg': 12 }
preserve = 0
for i in tqdm(range(10)):
    row = mg_df.iloc[i]
    """g_prod = nx.Graph(row["product_bonds"])
    g_react = nx.Graph(row["reactant_bonds"])
    g_prod_2 = nx.Graph(row["product_bonds_nometal"])
    g_react_2 = nx.Graph(row["reactant_bonds_nometal"])
    if(nx_graph_atom_mapping(g_prod, g_react) == None):
        if(nx_graph_atom_mapping(g_prod_2, g_react_2) == None):
            pass
        else: works_count += 1 
    else: works_count += 1"""
    species_reactant = [int_atom(i['name']) for i in row['reactant_molecule_graph']["molecule"]["sites"]]
    species_products = [int_atom(i['name']) for i in row['product_molecule_graph']["molecule"]["sites"]]
    coords_reactant = [i["xyz"] for i in row['reactant_molecule_graph']["molecule"]["sites"]]
    coords_products = [i["xyz"] for i in row['product_molecule_graph']["molecule"]["sites"]]
    #r = create_wrapper_mol_from_atoms_and_bonds(
    #    species_reactant, coords_reactant, row["reactant_bonds"], charge=0
    #)

    #p = create_wrapper_mol_from_atoms_and_bonds(
    #    species_products, coords_products, row["product_bonds"], charge=0
    #)
    reactant_mol = xyz2mol(atoms = species_reactant, 
    coordinates = coords_reactant
    )
    reactant_wrapper = rdkit_mol_to_wrapper_mol(reactant_mol[0],
        identifier = '1'
        )

    list_temp = [i['name'] for i in list(reactant_wrapper.mol_graph.as_dict()['molecule']['sites'])]
    list_xyz = [i['xyz'] for i in list(reactant_wrapper.mol_graph.as_dict()['molecule']['sites'])]
    list_atomic = [element_dict[i] for i in list_temp]
    
    if(list_atomic == species_reactant): preserve += 1
    if(list_xyz == coords_reactant): preserve += 1

print(preserve)


100%|██████████| 10/10 [00:00<00:00, 61.88it/s]

20


In [52]:
#len(rxn_alt_double_product.products[0].species)
#[{i:i} for i in range(len(rxn_alt_double_product.products[0].species))]

type(reactant_mol[0])
from rdkit.Chem.rdchem import Mol
